<a href="https://colab.research.google.com/github/Sillians/Basil-guide/blob/master/Dealing_with_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DEALING WITH MISSING VALUES**

**Introduction**

    There are many ways data can end up with missing values. For example,

    A 2 bedroom house won't include a value for the size of a third bedroom.
    A survey respondent may choose not to share his income.
    Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. So you'll need to choose one of the strategies below

In [0]:
import numpy as np
import pandas as pd

**Three Approaches**

1) A Simple Option: Drop Columns with Missing Values

        The simplest option is to drop columns with missing values
            Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

In [4]:
db = pd.read_csv('melb_data.csv')
db.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,Abbotsford,40 Federation La,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,Abbotsford,55a Park St,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0


In [0]:
from sklearn.model_selection import train_test_split

In [6]:
# Select Target
y = db.Price
y.head()

0    1480000.0
1    1035000.0
2    1465000.0
3     850000.0
4    1600000.0
Name: Price, dtype: float64

In [7]:
# To keep things simple, we'll use only numerical predictors
melb_predictors = db.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
X.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.7996,144.9984,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.8079,144.9934,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.8093,144.9944,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.7969,144.9969,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.8072,144.9941,4019.0


In [8]:
X.describe()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,13580.000000,13580.000000,13580.000000,13580.000000,13580.000000,13518.000000,13580.000000,7130.000000,8205.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,10.137776,3105.301915,2.914728,1.534242,1.610075,558.416127,151.967650,1964.684217,-37.809203,144.995216,7454.417378
std,0.955748,5.868725,90.676964,0.965921,0.691712,0.962634,3990.669241,541.014538,37.273762,0.079260,0.103916,4378.581772
min,1.000000,0.000000,3000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1196.000000,-38.182550,144.431810,249.000000
25%,2.000000,6.100000,3044.000000,2.000000,1.000000,1.000000,177.000000,93.000000,1940.000000,-37.856822,144.929600,4380.000000
50%,3.000000,9.200000,3084.000000,3.000000,1.000000,2.000000,440.000000,126.000000,1970.000000,-37.802355,145.000100,6555.000000
75%,3.000000,13.000000,3148.000000,3.000000,2.000000,2.000000,651.000000,174.000000,1999.000000,-37.756400,145.058305,10331.000000
max,10.000000,48.100000,3977.000000,20.000000,8.000000,10.000000,433014.000000,44515.000000,2018.000000,-37.408530,145.526350,21650.000000


In [9]:
# The feature varaibles ('Car', 'BuildingArea', and 'YearBuilt' all have missing Values)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 12 columns):
Rooms            13580 non-null int64
Distance         13580 non-null float64
Postcode         13580 non-null float64
Bedroom2         13580 non-null float64
Bathroom         13580 non-null float64
Car              13518 non-null float64
Landsize         13580 non-null float64
BuildingArea     7130 non-null float64
YearBuilt        8205 non-null float64
Lattitude        13580 non-null float64
Longtitude       13580 non-null float64
Propertycount    13580 non-null float64
dtypes: float64(11), int64(1)
memory usage: 1.2 MB


In [10]:
X.isnull().head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,False,False,False,False,False,False,False,True,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False


In [11]:
y.shape

(13580,)

In [0]:
# Divide Data into training and validation data set
X_train, X_val, y_train, y_val = train_test_split(X, y,test_size=0.2, random_state=0)

**Define Function to Measure Quality of Each Approach**

    We define a function score_dataset() to compare different approaches to dealing with missing values. This function reports the mean absolute error (MAE) from a random forest model.

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [0]:
# Function for comparing different approaches
def score_dataset(X_train, X_val, y_train, y_val):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_absolute_error(y_val, preds)

**Score from Approach 1 (Drop Columns with Missing Values)**

    Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.

In [18]:
# Get names of columns with missing valuee

col_with_missing = [col for col in X_train.columns
                   if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(col_with_missing, axis=1)
reduced_X_val   = X_val.drop(col_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_val, y_train, y_val))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


**2) A Better Option: Imputation**

    Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each column.

    The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

**Score from Approach 2 (Imputation)**

    Next, we use SimpleImputer to replace missing values with the mean value along each column.

    Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as regression imputation, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [30]:
from sklearn.impute import SimpleImputer

# Imputation
db_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(db_imputer.fit_transform(X_train))
imputed_X_val = pd.DataFrame(db_imputer.transform(X_val))


# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_val.columns  = X_val.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_val, y_train, y_val))

MAE from Approach 2 (Imputation):
178166.46269899711


We see that Approach 2 has lower MAE than Approach 1, so Approach 2 performed better on this dataset.

** An Extension To Imputation

    Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.
    
    In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.

**Score from Approach 3 (An Extension to Imputation)**

Next, we impute the missing values, while also keeping track of which values were imputed.

In [34]:
X_train_plus = X_train.copy()
X_val_plus  = X_val.copy()

# Make new columns indicating what will be imputed
for cols in col_with_missing:
    X_train_plus[cols + '_was_missing'] = X_train_plus[cols].isnull()
    X_val_plus[cols + '_was_missing'] = X_val_plus[cols].isnull()

# Imputation
new_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(new_imputer.fit_transform(X_train_plus))
imputed_X_val_plus  = pd.DataFrame(new_imputer.transform(X_val_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_val_plus.columns   = X_val_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_val_plus, y_train, y_val))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


As we can see, Approach 3 performed slightly worse than Approach 2.

**So, why did imputation perform better than dropping the columns?**

The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [36]:
# Shape of training data (num_rows, num_columns)

print(X_train.shape)

(10864, 12)


In [37]:
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


**Conclusion**

As is common, imputing missing values (in Approach 2 and Approach 3) yielded better results, relative to when we simply dropped columns with missing values (in Approach 1).